In [1]:
import requests, time, re
from bs4 import BeautifulSoup
import pandas as pd

from pykospacing import Spacing
from hanspell import spell_checker

# 리뷰 스크래핑

In [2]:
# 잡플래닛 로그인 url
url = "https://www.jobplanet.co.kr/users/sign_in?_nav=gb"
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'User-Agent':user_agent}
login_data = {'user':{'email':'hsmy31@hanyang.ac.kr', 'password':'rhkwpgksmswnd!', 'remember_me':'true'}}
session = requests.session()

# 로그인 실행
login_response = session.post(url, json = login_data, headers = headers)

reviews = []

In [3]:
# 1.정부/공공기관/공기업(1001), 2.은행/금융업(900), 3.네트워크/통신/모바일(704), 4.세무/회계(1004)
codes = ['1001', '900', '704', '1004']

In [19]:
# 리뷰를 txt파일로 저장
f = open('review_train.txt', 'w', encoding = 'utf-8')
reviews = {}

for code in codes:
    url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code)
    response =  session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    num = soup.find('span', class_='num') # 리뷰 개수 확인
    num = int(num.get_text().strip())
    import math
    pages = math.ceil(num / 10) # 한 페이지당 리뷰 10개로 구성됨
    
    for i in range(1, pages + 2): # 전체 페이지의 리뷰 추출
        time.sleep(1)
        url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code) + "&page="+ str(i)
        response =  session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        label = soup.find_all('h2', class_="us_label") # 한 줄 리뷰 추출
        
        # 추출한 리뷰에서 태그를 제외한 텍스트만 추출하여 labels 리스트에 저장
        labels = [label.get_text().replace('\r', ' ') for label in label[:]]   
        # 별점이 포함된 div 추출
        star = soup.find_all('div', class_="star_score") 
        # 추출한 div에서 별점을 나타내는 width 속성의 숫자로 된 부분을 stars 리스트에 저장
        stars = re.findall('[0-9]+[.]+[0-9]', str(star))
        
        # txt파일에 리뷰 쓰기
        for j in labels:
            f.write(j[5:-1] + '  ')
        
        # reviews 딕셔너리에 '리뷰: 별점' 형식으로 추가
        for k in range(len(labels)):
            reviews[labels[k][5:-1]] = stars[k].replace('.0','')
    f.write('\n')
            
f.close()

In [20]:
len(reviews)

2546

In [21]:
with open('review_train.txt', 'r', encoding = 'utf-8') as f:
    print(len(f.readlines()))

4


####  review_train.txt는 하나의 산업군 리뷰 전체를 한 줄로 스크래핑하여 총 4문장으로 이루어져 있음
 - 각 리뷰는 '  '로 구분됨
 - 이는 soynlp를 실행하는데 적합하도록 한 것임
 
#### 전체 산업군과 별도로 각각의 산업군별로 분석하기 위해 텍스트 파일을 4개로 분할


In [34]:
with open('review_train.txt', 'r', encoding = 'utf-8') as f :
    data = f.read().split('\n')
    for i in range(4):
        a = 1 + i
        with open(f'review_{a}.txt', 'w+', encoding = 'utf-8') as txt:
            txt.write(data[i]) 

# soynlp의 명사추출기 적용
#### - L + [R]의 bipartite graph의 정보를 이용하여 해당 단어가 명사인지 아닌지 판단
#### - 명사 가능 점수의 범위: [-1, 1]

In [2]:
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor

In [3]:
from soynlp.utils import DoublespaceLineCorpus

corpus_fname = 'review_train.txt'
sentences = DoublespaceLineCorpus(corpus_fname, iter_sent=True)
len(sentences)

2669

## 1. LRNounExtractor

In [4]:
from soynlp.noun import LRNounExtractor

noun_extractor = LRNounExtractor(
    max_left_length = 10, 
    max_right_length = 7,
    predictor_fnames = None,
    verbose = True
)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded


In [5]:
%%time
nouns = noun_extractor.train_extract(
    sentences,
    min_noun_score=0.3,
    min_noun_frequency=20
)

[Noun Extractor] scanning was done (L,R) has (612, 331) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 168 nouns are extracted
Wall time: 349 ms


In [6]:
# 빈도수 기준 상위 100개 단어
sort_nouns1 = sorted(nouns.items(), key=lambda x:-x[1].score)[:100]
print(sort_nouns1)

[('적당', NounScore_v1(frequency=23, score=1.0, known_r_ratio=0.13043478260869565)), ('굉장', NounScore_v1(frequency=20, score=1.0, known_r_ratio=0.05)), ('그만', NounScore_v1(frequency=23, score=1.0, known_r_ratio=0.045454545454545456)), ('다닐만', NounScore_v1(frequency=18, score=1.0, known_r_ratio=1.0)), ('처음', NounScore_v1(frequency=20, score=0.999943, known_r_ratio=1.0)), ('공공', NounScore_v1(frequency=2, score=0.999913, known_r_ratio=0.07142857142857142)), ('회계', NounScore_v1(frequency=5, score=0.999913, known_r_ratio=0.09090909090909091)), ('최악', NounScore_v1(frequency=21, score=0.9998325, known_r_ratio=0.8)), ('전반적', NounScore_v1(frequency=29, score=0.999725, known_r_ratio=0.5357142857142857)), ('보험', NounScore_v1(frequency=12, score=0.99972, known_r_ratio=0.2)), ('때문', NounScore_v1(frequency=26, score=0.99972, known_r_ratio=0.9615384615384616)), ('전반', NounScore_v1(frequency=2, score=0.99972, known_r_ratio=0.03333333333333333)), ('외국', NounScore_v1(frequency=3, score=0.999707, known_r_r

In [29]:
# 빈도수 기준 상위 100개 단어를 words_top100_fq.txt로 저장 및 출력
f = open('words_top100_fq.txt', 'w', encoding = 'utf-8')
for i, (word, score) in enumerate(sort_nouns1):
    f.write(sort_nouns1[i][0] + '  ')
    if i % 5 ==0:
        print()
    print('%6s (%.2f)' % (word, score.score), end = '')
f.close()


    적당 (1.00)    굉장 (1.00)    그만 (1.00)   다닐만 (1.00)    처음 (1.00)
    공공 (1.00)    회계 (1.00)    최악 (1.00)   전반적 (1.00)    보험 (1.00)
    때문 (1.00)    전반 (1.00)    외국 (1.00)    기관 (1.00)    서울 (1.00)
   커리어 (1.00)    지역 (1.00)    성과 (1.00)    분야 (1.00)   대부분 (1.00)
    다양 (1.00)    업계 (1.00)   수평적 (1.00)   안정적 (1.00)    사회 (1.00)
    대체 (1.00)    나름 (1.00)   보수적 (1.00)    지방 (1.00)   수직적 (1.00)
    전문 (1.00)   있는곳 (1.00)    편안 (1.00)    관련 (1.00)    싶은 (1.00)
    확실 (1.00)    무난 (1.00)    중요 (1.00)    비추 (1.00)    계속 (1.00)
    부서 (1.00)    부족 (0.99)    필요 (0.99)    가능 (0.99)    입사 (0.99)
    기대 (0.99)    추천 (0.99)    꼰대 (0.99)    자유 (0.99)    만족 (0.99)
   외국계 (0.98)    구조 (0.98)    좋아 (0.98)    함께 (0.98)    최고 (0.96)
   시스템 (0.96)  기업문화 (0.96)   마인드 (0.96)    분들 (0.96)    본인 (0.95)
    은행 (0.94)   출퇴근 (0.94)    기회 (0.93)    생각 (0.93)    운영 (0.93)
    느낌 (0.93)  중소기업 (0.92)    복지 (0.92)   사람들 (0.92)    노력 (0.92)
    신입 (0.91)    도움 (0.90)    개인 (0.90)    위치 (0.90)    규모 (0.89)
    체계 (0

In [32]:
# 빈도수 * 명사점수 기준 상위 100개 단어
sort_nouns2 = sorted(nouns.items(), key=lambda x:-x[1].frequency * x[1].score)[:100]
print(sort_nouns2)

[('회사', NounScore_v1(frequency=597, score=0.6511768373493976, known_r_ratio=0.7685185185185185)), ('업무', NounScore_v1(frequency=253, score=0.8790694967320261, known_r_ratio=0.9386503067484663)), ('다양', NounScore_v1(frequency=139, score=0.9994220367647055, known_r_ratio=0.9784172661870504)), ('기업', NounScore_v1(frequency=151, score=0.8827037500000001, known_r_ratio=0.5625)), ('추천', NounScore_v1(frequency=115, score=0.9883593124999999, known_r_ratio=0.32)), ('사람들', NounScore_v1(frequency=120, score=0.9160305, known_r_ratio=1.0)), ('생각', NounScore_v1(frequency=117, score=0.9327781363636364, known_r_ratio=0.5892857142857143)), ('복지', NounScore_v1(frequency=110, score=0.9164357058823529, known_r_ratio=1.0)), ('사람', NounScore_v1(frequency=172, score=0.5357988633540373, known_r_ratio=0.6313725490196078)), ('안정적', NounScore_v1(frequency=80, score=0.999177525, known_r_ratio=0.5194805194805194)), ('직원들', NounScore_v1(frequency=90, score=0.8537492068965518, known_r_ratio=0.9508196721311475)), ('경

In [33]:
# 빈도수 * 명사점수 기준 상위 100개 단어를 words_top100_sc.txt로 저장 및 출력
f = open('words_top100_sc.txt', 'w', encoding = 'utf-8')
for i, (word, score) in enumerate(sort_nouns2):
    f.write(sort_nouns2[i][0] + '  ')
    if i % 5 ==0:
        print()
    print('%6s (%.2f)' % (word, score.score), end = '')
f.close()


    회사 (0.65)    업무 (0.88)    다양 (1.00)    기업 (0.88)    추천 (0.99)
   사람들 (0.92)    생각 (0.93)    복지 (0.92)    사람 (0.54)   안정적 (1.00)
   직원들 (0.85)    경험 (0.60)    직장 (0.80)    문화 (0.83)    성장 (0.81)
    체계 (0.88)    하는 (0.65)    급여 (0.88)    가능 (0.99)    근무 (0.69)
    연봉 (0.62)    자유 (0.99)    본인 (0.95)    경력 (0.78)    최고 (0.96)
   보수적 (1.00)    신입 (0.91)    조직 (0.88)    좋아 (0.98)    직원 (0.48)
    무난 (1.00)    환경 (0.74)    싶은 (1.00)   공무원 (0.79)   커리어 (1.00)
    나름 (1.00)    이상 (0.82)   계약직 (0.67)  공공기관 (0.86)   수평적 (1.00)
    부서 (1.00)    개인 (0.90)   워라벨 (0.64)    노력 (0.92)    기회 (0.93)
    입사 (0.99)    대표 (0.84)    함께 (0.98)   전반적 (1.00)    업계 (1.00)
   있는곳 (1.00)    가족 (0.88)    느낌 (0.93)    분들 (0.96)    위치 (0.90)
    대우 (0.71)    규모 (0.89)   사무실 (0.82)   공기업 (0.87)    부족 (0.99)
    필요 (0.99)    발전 (0.50)    운영 (0.93)    때문 (1.00)    계속 (1.00)
    장점 (0.59)   워라밸 (0.43)    기관 (1.00)    야근 (0.77)    확실 (1.00)
    시간 (0.72)    은행 (0.94)    적당 (1.00)    그만 (1.00)    전문 (1.00)
    비추 (1

## 2. NewsNounExtractor

In [50]:
from soynlp.noun import NewsNounExtractor

noun_extractor = NewsNounExtractor(
    max_left_length=10, 
    max_right_length=7,
    predictor_fnames=None,
    verbose=True
)

used default noun predictor; Sejong corpus based logistic predictor
C:/Users/hsmy1/Anaconda3/envs/py37/lib/site-packages/soynlp
local variable 'f' referenced before assignment
local variable 'f' referenced before assignment


In [51]:
nouns = noun_extractor.train_extract(sentences)

scan vocabulary ... 
done (Lset, Rset, Eojeol) = (26563, 15906, 13021)
predicting noun score was done                                        
before postprocessing 3516
_noun_scores_ 1086
checking hardrules ... done / 1086떡볶+(이)), NVsubE (사기(당)+했다) ... done
after postprocessing 668
extracted 0 compounds from eojeols

In [60]:
score = nouns[word]
word = '워라벨'
print('word: %s, score: %.2f, frequency: %.2f' % (word, score.score, score.frequency))
#('%6s (%.2f)' % (word, score.score), end = '')

word: 워라벨, score: 0.45, frequency: 61.00


### 각 산업군별 리뷰 학습

In [84]:
corpus_fname1 = 'review_1.txt'
sentences1 = DoublespaceLineCorpus(corpus_fname1, iter_sent=True)
len(sentences1)

812

In [41]:
corpus_fname2 = 'review_2.txt'
sentences2 = DoublespaceLineCorpus(corpus_fname2, iter_sent=True)
len(sentences2)

1113

In [42]:
corpus_fname3 = 'review_3.txt'
sentences3 = DoublespaceLineCorpus(corpus_fname3, iter_sent=True)
len(sentences3)

358

In [43]:
corpus_fname4 = 'review_4.txt'
sentences4 = DoublespaceLineCorpus(corpus_fname4, iter_sent=True)
len(sentences4)

386

In [85]:
def nounextractor(sentences):
    nouns = noun_extractor.train_extract(
        sentences,
        min_noun_score=0.3,
        min_noun_frequency=15
    )

    # 빈도수 * 명사점수 기준 상위 100개 단어
    sort_nouns = sorted(nouns.items(), key=lambda x:-x[1].frequency * x[1].score)[:100]

    for i, (word, score) in enumerate(sort_nouns):
        if i % 5 ==0:
            print()
        print('%6s (%.2f, %.2f)' % (word, score.frequency, score.score), end = '')

In [86]:
nounextractor(sentences)

[Noun Extractor] scanning was done (L,R) has (792, 434) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 220 nouns are extracted

    회사 (598.00, 0.65)    업무 (259.00, 0.88)    다양 (139.00, 1.00)    기업 (152.00, 0.89)    추천 (115.00, 0.99)
    생각 (119.00, 0.93)   사람들 (121.00, 0.92)    복지 (111.00, 0.92)    사람 (178.00, 0.55)   직원들 (96.00, 0.85)
   안정적 (80.00, 1.00)    경험 (112.00, 0.60)    직장 (83.00, 0.80)    성장 (77.00, 0.81)    문화 (74.00, 0.83)
    급여 (66.00, 0.89)    하는 (88.00, 0.65)    가능 (56.00, 0.99)    근무 (79.00, 0.70)    연봉 (81.00, 0.62)
    자유 (51.00, 0.99)    본인 (52.00, 0.95)    최고 (51.00, 0.96)    체계 (55.00, 0.88)    경력 (62.00, 0.78)
    조직 (52.00, 0.89)    신입 (48.00, 0.92)   보수적 (44.00, 1.00)    좋아 (44.00, 0.98)   공무원 (52.00, 0.82)
    직원 (86.00, 0.48)    무난 (40.00, 1.00)    환경 (53.00, 0.74)    싶은 (39.00, 1.00)   커리어 (37.00, 1.00)
  공공기관 (41.00, 0.87)    나름 (35.00, 1.00)   계약직 (51.00, 0.68)   수평적 (34.00, 1.00)    이상 (41.00, 0.82)
    부서 (33.00, 1.00)    개인 (36.0

In [87]:
nounextractor(sentences1)

[Noun Extractor] scanning was done (L,R) has (250, 128) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 50 nouns are extracted

    업무 (95.00, 0.84)   분위기 (58.00, 1.00)    다양 (42.00, 1.00)    생각 (37.00, 0.88)   공무원 (42.00, 0.77)
    직장 (37.00, 0.86)    복지 (32.00, 0.97)   계약직 (37.00, 0.81)    근무 (35.00, 0.85)   사람들 (34.00, 0.86)
   안정적 (29.00, 1.00)  공공기관 (34.00, 0.82)    추천 (27.00, 0.99)    경험 (43.00, 0.60)    회사 (67.00, 0.34)
    기업 (23.00, 1.00)    급여 (23.00, 0.92)    기관 (21.00, 1.00)   공기업 (21.00, 0.94)    가능 (20.00, 0.99)
    조직 (20.00, 0.88)    사람 (44.00, 0.39)   직원들 (19.00, 0.87)    환경 (21.00, 0.78)    서울 (16.00, 1.00)
   워라밸 (24.00, 0.62)    체계 (19.00, 0.76)    좋아 (14.00, 0.98)    자유 (14.00, 0.96)   정규직 (18.00, 0.73)
    본인 (15.00, 0.87)    문화 (18.00, 0.71)    위치 (15.00, 0.83)    무난 (12.00, 1.00)    만족 (11.00, 1.00)
    장점 (19.00, 0.57)    사회 (10.00, 1.00)    사업 (16.00, 0.61)    경력 (11.00, 0.85)    민원 (11.00, 0.79)
    연구 (9.00, 0.94)    눈치 (10.00, 0.79)    

In [81]:
nounextractor(sentences2)

[Noun Extractor] scanning was done (L,R) has (248, 139) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 52 nouns are extracted

    회사 (315.00, 0.64)    업무 (83.00, 0.92)    기업 (75.00, 0.83)   사람들 (53.00, 0.93)    다양 (47.00, 1.00)
    생각 (44.00, 0.99)    사람 (54.00, 0.80)    추천 (43.00, 0.98)    하는 (39.00, 1.00)   안정적 (37.00, 1.00)
    연봉 (54.00, 0.67)    복지 (42.00, 0.86)    문화 (41.00, 0.88)    성장 (44.00, 0.81)   직원들 (41.00, 0.78)
    최고 (30.00, 0.93)   보수적 (27.00, 1.00)    직장 (32.00, 0.80)    본인 (26.00, 0.98)    체계 (25.00, 0.97)
    경험 (37.00, 0.63)    직원 (38.00, 0.59)    무난 (22.00, 1.00)    급여 (26.00, 0.83)    은행 (23.00, 0.94)
   커리어 (21.00, 1.00)    투자 (21.00, 0.99)    규모 (22.00, 0.88)    대표 (22.00, 0.87)    업계 (18.00, 1.00)
    근무 (23.00, 0.76)    조직 (20.00, 0.88)    이상 (21.00, 0.83)    경력 (24.00, 0.73)   외국계 (16.00, 0.98)
    좋아 (16.00, 0.97)    성과 (15.00, 1.00)    대우 (21.00, 0.70)    장점 (18.00, 0.80)    운영 (15.00, 0.94)
    가능 (14.00, 0.99)    발전 (29.00, 0.40)  

In [88]:
nounextractor(sentences3)

[Noun Extractor] scanning was done (L,R) has (97, 58) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 14 nouns are extracted

    회사 (124.00, 0.84)    기업 (38.00, 0.86)    업무 (32.00, 0.80)    다양 (20.00, 1.00)    복지 (16.00, 0.95)
   직원들 (12.00, 0.92)    생각 (12.00, 0.90)    직원 (14.00, 0.70)    성장 (14.00, 0.69)    경험 (14.00, 0.57)
    사람 (18.00, 0.37)    사업 (9.00, 0.55)    근무 (7.00, 0.70)    연봉 (12.00, 0.40)

In [89]:
nounextractor(sentences4)

[Noun Extractor] scanning was done (L,R) has (115, 60) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 16 nouns are extracted

    회사 (75.00, 0.55)    업무 (39.00, 0.93)    다양 (28.00, 1.00)    사람 (35.00, 0.77)    신입 (21.00, 0.99)
   사무실 (20.00, 1.00)   사람들 (18.00, 0.96)   직원들 (15.00, 0.95)   회계사 (13.00, 0.97)    야근 (11.00, 1.00)
   거래처 (14.00, 0.74)    경력 (14.00, 0.70)    입사 (8.00, 0.99)    경험 (13.00, 0.54)   세무사 (11.00, 0.46)
    개인 (5.00, 1.00)